In [1]:
import numpy as np
from tqdm import tqdm
import cv2

n = 25000
img_width = 299
img_height = 299

X = np.zeros((n,img_width,img_height,3),dtype=np.uint8)
y = np.zeros((n,1),dtype = np.uint8)

for i in tqdm(range(n//2)):
    X[i] = cv2.resize(cv2.imread('train/cat.%d.jpg' %i),(img_width,img_height))
    X[i+n//2] = cv2.resize(cv2.imread('train/dog.%d.jpg' %i),(img_width,img_height))

y[n//2:] = 1

100%|██████████| 12500/12500 [00:51<00:00, 242.82it/s]


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

In [3]:
from keras.models import *
from keras.applications import *
from keras.layers import *
from keras.preprocessing import *

Using TensorFlow backend.


In [4]:
base_model = InceptionV3(input_tensor=Input((img_width,img_height,3)),weights='imagenet',include_top=False,pooling='avg')

In [5]:
# len(base_model.layers) - 1
NB_LAYER_FREEZE = len(base_model.layers) - 1
for layer in base_model.layers[:NB_LAYER_FREEZE]:
    layer.trainable = False
for layer in base_model.layers[NB_LAYER_FREEZE:]:
    layer.trainable = True
    
x = Dropout(0.25)(base_model.output)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)

In [6]:
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
model.fit(X_train,y_train, batch_size=16, epochs=10, validation_data=(X_valid,y_valid))

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 133s - loss: 0.2130 - acc: 0.9294 - val_loss: 0.1109 - val_acc: 0.9666
Epoch 2/10
20000/20000 [==============================] - 133s - loss: 0.1380 - acc: 0.9496 - val_loss: 0.0912 - val_acc: 0.9702
Epoch 3/10
20000/20000 [==============================] - 133s - loss: 0.1238 - acc: 0.9540 - val_loss: 0.0821 - val_acc: 0.9736
Epoch 4/10
20000/20000 [==============================] - 132s - loss: 0.1192 - acc: 0.9547 - val_loss: 0.0793 - val_acc: 0.9712
Epoch 5/10
20000/20000 [==============================] - 132s - loss: 0.1211 - acc: 0.9535 - val_loss: 0.0761 - val_acc: 0.9726
Epoch 6/10
20000/20000 [==============================] - 132s - loss: 0.1165 - acc: 0.9559 - val_loss: 0.0709 - val_acc: 0.9752
Epoch 7/10
20000/20000 [==============================] - 132s - loss: 0.1127 - acc: 0.9564 - val_loss: 0.0746 - val_acc: 0.9722
Epoch 8/10
20000/20000 [========================

In [8]:
model.save('models/inception3_fine_tune.h5')